# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2"
HOME_DIR = "/media/notebooks/Assignments/HW2" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [3]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [4]:
# data path
ENRON = "data/enronemail_1h.txt"

# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Hadoop MapReduce is based on the funtional programming paradigm. The main ideas of functional programming paradigm is that higher order function that takes another function as arguments. The map in MapReduce exemplifies a function that takes a transformation logic (which is a function) and a list as inputs and apply the transformation logic on each of the elements in the input list. The reduce in MapReduce exemplifies a function that takes a aggregation logic (which is another function) and a ordered list as an input and apply the logic on all the inputs of the same keys.

> __b)__ Hadoop Shuffle is the tranfers of the output of map tasks to the reduce tasks as input. It might be costly because the system might hit network bottleneck when data has to be transferred from the nodes where map tasks are to the nodes where reduce tasks reside. Plus this also involves reading and writing to disk and this adds more cost in terms of time. One thing we can do is to utilize combiners to make our key-value pairs more compact and reduce the volume of the data we need to read, copy, transfer and write between mappers and reducers.

> __c)__ Because when using combiners, it is the framework that has control over how many times and even whether the combiners are going to be used at all. Therefore, it is not guaranteed that the reducers will receive data that is in the same format as the combiners output. Therefore, both combiners and mappers need to have the same output format that is the format the reducers would like to take as input.  

> __d)__ In the mapping stage, developers can only hint the framework on the extent of the parallelization (i.e., how many mappers to use); however, it is ultimately decided by the framework how many mappers are going to be deployed depending on factors such as the location of the data, and etc... On the other hand, the developers have control over the exact number of reducers used in the job. 

> __e)__ The increasing availability of the amount of data for people to draw insight from and the commodity-hardware that can form as a cluster of machines have prompted the creation of parallel computation frameworks like hadoop.

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ Counters give different information about our Hadoop Streaming jobs. They are useful because we can use them to understand how our job was executed, debug and optimize our job. By default, Hadoop provides File System Counters (about the files a Hadoop job take in and output) and Job Counters (about our map tasks and reduce tasks). Custom counters can be created by inluding the following line of code `sys.stderr.write("reporter:counter:MyWordCounters,counter-name,10\n")`. This will create a custom counter, _counter-name_ under group, _MyWordCounters_ with a count of 10. This line of code can be placed in mapper, combiner or reducer scripts. In addition, just like a regular counter, the counts will be outputted to the console.

> __b)__ A composite key is a key made by appending to our original key with a delimiter another unit that is meaningful in the context of our Hadoop job on its own. For example, we can form a composite key by including the value in a key-value pair. They are useful because since the key is used for both partitioning and sorting, a composite key will enable the job with more options for sorting and partitioning. For example, when I create a composite key by including the value in a key-value pair. I can partition by the original key and sort by the value. The custom partitioning can be achieve with the use of composite key. One can explicitly assign a value to be included as the composite key of a key-value pair and have hadoop parition by the value. 

> __c)__ The order inversion pattern is a design pattern where through proper design we can access the result of some calculation in the reducers before processing the records needed for that calculation. In the count frequency problem, the order inversion design pattern enable us get the total word count (the result of we can get from aggregating all the records) first before any words were processed by the reducers. Therefore, we could divide the count of a word by the total word count to get the frequency of a word as the reducers finish processing a word. We implemented it by customize a key that will guarantee all the records corresponding to that key will be pushed to the top in the sorting results (in the case of using multiple reducers, we will also need to use custom partitioning to make sure each reducer is getting a total count).   


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ A partial sort is the result of sorting operation where records within files are sorted but not across files. An unordered total sort is a result of sorting where records across files are sorted (i.e., all keys used to sort in one files are either bigger or smaller than those in another file) but the sequence of files are not aligned with the sorted results (i.e., not all the keys used to sort in the previous file are bigger than all the keys used to sort in the current file in the case of descending sort). The total order sort is a result of the sorting where records are sorted within and acorss files, and the files are ordered in way that is aligned with the objective of the sorting operation. Total order sort will enable us to ready and process the records in a sequence of files sequentially without having to keep track of the state. We can look up the top or bottom records without having to scan through all the files. Such desirable properties come with the result of a total order sort. The cost to work with unordered sort becomes very great as the datasets get large. 

> __b)__ Partitioning phase of MapReduce needs to be leveraged to implement Total Order Sort. We need to change the default behavior of the hashing so that the partition keys are assigned taking into consideration of its position amoung all other partitions. It needs to be changed here because this is where the sequence of the files is determined.

> __c)__ We need to tell the system that we are going to use composite keys. We need to tell Hadoop that we are using the keys in the composite key for partitioning and sorting. We need to tell Hadoop that We need to specify which key(s) we are using for partitioning and which keys(s) we are using for sorting. 

> __d)__ To order the partition keys so that when keys for sorting are used to generate a partition key, they are assigned with partition keys that will guarantee them to be sorted across partitions.

> __e)__ In the mapper script.

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [5]:
!pwd

/media/notebooks/Assignments/HW2


In [6]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 /media/notebooks/Assignments/HW2/{ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [7]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l /media/notebooks/Assignments/HW2/{ENRON}

100 /media/notebooks/Assignments/HW2/data/enronemail_1h.txt


In [8]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

In [9]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal /media/notebooks/Assignments/HW2/{ENRON} {HDFS_DIR}/enron.txt

In [10]:
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root supergroup     204559 2019-09-18 21:11 /user/root/HW2/enron.txt


# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in real emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output of your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected then run the provided Hadoop streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Briefly explain why or why not.


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Edit your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ The word asssistance occurred twice in the ham class and eight times in the spam class.

> __c)__ We are only aggregating the counts of words as the reducers process the data. Therefore, it would have been impossible to sort the data on the result of the reduce tasks.  

> __d)__ I will use only two reducers and try to partition the key-value pairs by the is_spam field, then print the top ten records of each output file from the sorted results of the reducers.

In [11]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [12]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [13]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	1	1
one	0	2
two	1	0
two	0	1


In [14]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

rm: `/user/root/HW2/eda-output': No such file or directory


In [15]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob460403120325841111.jar tmpDir=null
19/09/18 19:59:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 19:59:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 19:59:39 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 19:59:39 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 19:59:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568836755256_0001
19/09/18 19:59:41 INFO impl.YarnClientImpl: Submitted application application_1568836755256_0001
19/09/18 19:59:41 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568836755256_0001/
19/09/18 19:59:41 INFO mapreduce.Job: Running job: job_1568836755256_0001
19/09/18 19:59:50 INFO mapreduce.Job: Job job_1568836755256_0001 running in uber mode : false
19/09/18 19:59:50 INFO mapreduce.Job:  map 0% reduce 0%
19/09/

In [16]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [17]:
# part b - write your grep command here
!grep assistance EnronEDA/results.txt

assistance	1	8
assistance	0	2


In [18]:
# part d/e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

rm: `/user/root/HW2/eda-sort-output': No such file or directory


In [19]:
# part d/e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k3,3nr" \
  -D mapreduce.partition.keypartitioner.options="-k2,2" \
  -mapper /bin/cat \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob3614549535295051912.jar tmpDir=null
19/09/18 20:00:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 20:00:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 20:00:17 INFO mapred.FileInputFormat: Total input paths to process : 2
19/09/18 20:00:17 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 20:00:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568836755256_0002
19/09/18 20:00:17 INFO impl.YarnClientImpl: Submitted application application_1568836755256_0002
19/09/18 20:00:17 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568836755256_0002/
19/09/18 20:00:17 INFO mapreduce.Job: Running job: job_1568836755256_0002
19/09/18 20:00:24 INFO mapreduce.Job: Job job_1568836755256_0002 running in uber mode : false
19/09/18 20:00:24 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [20]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ 31490 records is the number of the output of mappers and the input of the reducers. According to the mapper code, since we are emitting one record per word, so this means the total number of words we have in our dataset.

> __c)__ The number of records emitted by mappers is still 31490; however, the number of reducer input records has dropped down to 13096. This is the work done by the combiners. The combiners combined records that have the same keys.

> __d)__ An obvious scenario is when we don't have any records that share the same key in a given partition. Another case would be, when each mapper is only dealing with very few records, it might be more likely to not have any records having the same key in the same parition and also to not have enough spills for combiners to do any work before the shuffle stage. 

In [21]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [22]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob1644425443111381626.jar tmpDir=null
19/09/17 20:11:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/17 20:11:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/17 20:11:49 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/17 20:11:49 INFO mapreduce.JobSubmitter: number of splits:2
19/09/17 20:11:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568751006211_0003
19/09/17 20:11:49 INFO impl.YarnClientImpl: Submitted application application_1568751006211_0003
19/09/17 20:11:49 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568751006211_0003/
19/09/17 20:11:49 INFO mapreduce.Job: Running job: job_1568751006211_0003
19/09/17 20:11:56 INFO mapreduce.Job: Job job_1568751006211_0003 running in uber mode : false
19/09/17 20:11:56 INFO mapreduce.Job:  map 0% reduce 0%
19/09

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__ $\hat P(c|d) = \hat P(c) \prod_{1\leq k\leq n_d} \hat P(t_k|c)$. According to the math, we will need the estimated prior probability of a class, c and the estimated conditional probability of term $t_k$ of class c ($t_k$ represents one of the terms in the document, d that we are using for the classification task. There are usually multiple terms a model use, so all the conditional probability estimates of different terms needed to be calculated from the training set).

> __b)__ $\hat P(c)$ is the prior probability of a class which means how prevalent the class is in the training set. It's calculated by the number of document in a class devided by the total number of documents in the training set. $\hat P(t_k|c)$ is the estimated conditional probability of term $t_k$ of class c which means how often the term occurs relative to other terms for a given class, c. It is calculated by the count of a term divdied by the total count of all terms in a given class, c in the training set. We will only need one pass on the data to retrieve this information for all classes and all words.

> __c)__ 

>> $\hat P(1) = 3/4 =\frac{3}{4}$  
>> $\hat P(Chinese|1) = 5/8 =\frac{5}{8}$  
>> $\hat P(Beijing|1) = \hat P(Shanghai|1) = \hat P(Macao|1) = 1/8 = \frac{1}{8}$  
>> $\hat P(Tokyo|1) = \hat P(Japan|1) = 0/8 = 0$  

>> $\hat P(0) = 1/4 =\frac{1}{4}$  
>> $\hat P(Chinese|0) = 1/3 =\frac{1}{3}$  
>> $\hat P(Beijing|0) = \hat P(Shanghai|0) = \hat P(Macao|0) = 0/3 = 0$  
>> $\hat P(Tokyo|0) = \hat P(Japan|0) = 1/3 = \frac{1}{3}$  

> __d)__ 
>> d = "Chinese Chinese Chinese Tokyo Japan" 

>> $\hat P(1|d) = \hat P(1) \cdot \hat P(Chinese|1) \cdot \hat P(Chinese|1) \cdot \hat P(Chinese|1) \cdot \hat P(Tokyo|1) \cdot \hat P(Japan|1)$  
$= \frac{3}{4} \cdot \frac{5}{8} \cdot \frac{5}{8} \cdot \frac{5}{8} \cdot 0 \cdot 0 = 0$ 

>> $\hat P(1|d) = \hat P(0) \cdot \hat P(Chinese|0) \cdot \hat P(Chinese|0) \cdot \hat P(Chinese|0) \cdot \hat P(Tokyo|0) \cdot \hat P(Japan|0)$  
$= \frac{1}{4} \cdot \frac{1}{3} \cdot \frac{1}{3} \cdot \frac{1}{3} \cdot \frac{1}{3} \cdot \frac{1}{3} = \frac{1}{972}$ 

>> The model will classify the document as "not about China" since that is the class which has the higher probability

> __e)__ Smoothing makes our model more resilient to overfitting. Our unsmoothed model classified the document, "Chinese Chinese Chinese Tokyo Japan" as not about China simply because Tokyo and Japan didn't occur in a documnet associated with the label, "about China" in our training set. The estimates of the conditional probablities, $\hat P(Tokyo|1), \hat P(Japan|1)$ were zero and that caused the estimated probability, $\hat P(1|d)$ to be zero. This gave the two terms unreasonably large influence over the predicitons of the model. In contrast the smoothed model was able to lower the influence of the absence of terms like Tokyo and Japan of a given label by assigning a default occurance of one to all terms, which in turns, allows the model to make dicisions based on more important evidence in our documnet such as term, "China". As a result, the model was able to classify documents more reasonably.



In [ ]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ 
>> The keys are the first value in the records, and the values are the information in all the rest of each record.
>> The record having its key as ClassPriors is different. It contains the prior probabilities for the classes  
>> The first two numbers of each records are not needed to make inferences. They represent the counts of term occurances given a label, or in the case of ClassPriors, the count of documents given a label.

> __b)__ Because when we multiply many fractions, in computation, the number might get really small for us to run into underflow issue. Therefore, we perform addition for the logs of probabilities instead. If we have a unsmoothed model, some estimates for conditional probabilities might be zero, and the log of zero is undefined. We might encounter this error when taking logs on the probability estimates when using an unsmoothed model. 

> __c)__ We disregard it by assigning a conditional probablity of 1 or 0 for the log of conditional probability such that they don't influence the outcome of the inference.

> __d)__ 
>> I don't think loading the model file into memory on the nodes where we do our mapping goes against the functional programming principle bacause the mapper nodes only read from the model file and use it for subsequent computation and do not update the file. Therefore, the mappers do not need to be aware of anything outside of their tast as they perform the computation which fits the principles of functional programming.  
>> From scalability point of view, when having a very lean model with a small vocabulary like this example, it works to load to memory. However, when working with model that might need to use a big vocabulary, it would be unwise to design it to load all the terms and conditional probabilities into memory.

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ I will just have the mappers calculate the partial sum of log probabilities for the parts of different documents they are assigned with. They should emit as output docID, \_class as keys and logpHam, logpSpam as values. The reducers will aggregate logpHam and logpSpam by the key and predict which class a document is.

Run these cells to create the example corpus and model.

In [23]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Writing NaiveBayes/chineseTrain.txt


In [24]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Writing NaiveBayes/chineseTest.txt


In [25]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Writing NBmodel.txt


In [11]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

Your work for `part e` starts here:

In [27]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [31]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [42]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

Deleted /user/root/HW2/chinese-output


In [43]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8204806717041595072.jar tmpDir=null
19/09/17 21:07:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/17 21:07:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/17 21:07:47 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/17 21:07:47 INFO mapreduce.JobSubmitter: number of splits:2
19/09/17 21:07:47 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568751006211_0006
19/09/17 21:07:47 INFO impl.YarnClientImpl: Submitted application application_1568751006211_0006
19/09/17 21:07:47 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568751006211_0006/
19/09/17 21:07:47 INFO mapreduce.Job: Running job: job_1568751006211_0006
19/09/17 21:07:54 INFO mapreduce.Job: Job job_1568751006211_0006 running in uber mode : false
19/09/17 21:07:54 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [44]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [45]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 5.


* __c) short response:__ We saw in Question 6 that adding Laplace +1 smoothing makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are a couple of approaches that we could take to handle the extra piece of information you identified in `c`: 1) if we knew this extra information beforehand, we could provide it to our reducer as a configurable parameter for the vocab size dynamically (_where would we get it in the first place?_). Or 2) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. 


* __e) code + short response:__ Choose one of the 2 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). [`HINT:` _don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications_].


__IMPORTANT NOTE:__ For full credit on this question, your code must work with multiple reducers. [`HINT:`_You will need to implement custom partitioning - [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb)]


### Q8 Student Answers:
> __ c)__ We will need the number of unique words in our training set

> __ d)__
>> 1) We can run another pre-processing MapReduce job before training our model to compute the total number of unique words in our training set. It's very simple to do but this would also mean that we will have to take two passes on the entire training set to train the model with smoothing.
>> 2) We can also produce the unique word count as we are processing our sorted word list in the reducers by just increment the count whenever we encounter a new word. However, we will have to post-process to get the smoothed NB model because the total number of unique words of our training set will be the sum of the total numbers of unique words in each partition. We won't be able to obtain this piece of information since this is part of the result of our entire MapReduce job. The implementation of this approach will require us to write the entire vocabulary to disk as a intermediary step which is a lot more than approach 1) which only wrote counts and read entire vocabulary again to calculate the smoothed conditional probabilities.

> __ e)__ I will choose the latter of the two options. Since the size of the vocabulary is usually orders of magnitude smaller than the original text it is extracted from. I'm thinking the latter approach might work better for really large datasets.

In [12]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 NaiveBayes/train_reducer.py | tail -n 6

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
INPUT:                                                    
    DocID \t true_class \t subject \t body                
OUTPUT:                                                   
    partitionKey \t word \t class0_partialCount,class1_partialCount       
    
=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    partitionKey \t word \t class0_partialCount,class1_partialCount  
OUTPUT:
    word \t class0_totalCount,class1_totalCount,class0_conditionalProbability,class1_conditionalProbability


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [13]:
# part b - write a unit test for your mapper here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py | column -t

A  chinese      0,1
A  beijing      0,1
A  chinese      0,1
A  chinese      0,1
A  chinese      0,1
A  shanghai     0,1
A  chinese      0,1
A  macao        0,1
A  tokyo        1,0
A  japan        1,0
A  chinese      1,0
A  *docTotals   1,3
A  *wordTotals  3,8


In [14]:
# part b - write a unit test for your reducer here
!echo -e "A	title	0,1\nA	body	0,1\nA	title	1,0\nA	body	1,0\nA	*docTotals	1,1\nA	*wordTotals	2,2"| sort -k1,1 | NaiveBayes/train_reducer.py |column -t

ClassPriors  1.0,1.0,0.5,0.5
body         1.0,1.0,0.5,0.5
title        1.0,1.0,0.5,0.5


In [15]:
# part b - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py| sort -k1,1 | NaiveBayes/train_reducer.py | column -t

ClassPriors  1.0,3.0,0.25,0.75
beijing      0.0,1.0,0.0,0.125
chinese      1.0,5.0,0.3333333333333333,0.625
japan        1.0,0.0,0.3333333333333333,0.0
macao        0.0,1.0,0.0,0.125
shanghai     0.0,1.0,0.0,0.125
tokyo        1.0,0.0,0.3333333333333333,0.0


In [16]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothed-chinese-NB-model

rm: `/user/root/HW2/unsmoothed-chinese-NB-model': No such file or directory


In [17]:
# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/unsmoothed-chinese-NB-model \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob1240893402786393711.jar tmpDir=null
19/09/18 21:12:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 21:12:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 21:12:40 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 21:12:40 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 21:12:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568841085575_0001
19/09/18 21:12:41 INFO impl.YarnClientImpl: Submitted application application_1568841085575_0001
19/09/18 21:12:41 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568841085575_0001/
19/09/18 21:12:41 INFO mapreduce.Job: Running job: job_1568841085575_0001
19/09/18 21:12:49 INFO mapreduce.Job: Job job_1568841085575_0001 running in uber mode : false
19/09/18 21:12:49 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [18]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/unsmoothed-chinese-NB-model/part-000* > NaiveBayes/unsmoothed-chinese-NB-modelResults.txt

In [19]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/unsmoothed-chinese-NB-modelResults.txt | column -t

ClassPriors  1.0,3.0,0.25,0.75
beijing      0.0,1.0,0.0,0.125
japan        1.0,0.0,0.3333333333333333,0.0
tokyo        1.0,0.0,0.3333333333333333,0.0
ClassPriors  1.0,3.0,0.25,0.75
chinese      1.0,5.0,0.3333333333333333,0.625
macao        0.0,1.0,0.0,0.125
shanghai     0.0,1.0,0.0,0.125


In [20]:
!cat NaiveBayes/chineseTrain.txt

D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [21]:
# part e - write a unit test for your NEW reducer here
!chmod a+x NaiveBayes/train_reducer_smooth.py
!chmod a+x NaiveBayes/postprocess_mapper.py
!chmod a+x NaiveBayes/postprocess_reducer.py
!echo -e "A	title	0,1\nA	body	0,1\nA	title	1,0\nA	body	1,0\nA	*docTotals	1,1\nA	*wordTotals	2,2"| sort -k1,1 | NaiveBayes/train_reducer_smooth.py |column -t

ClassPriors    1.0,1.0,0.5,0.5
*total_count   2.0,2.0,0.0,0.0
body           1.0,1.0,0.5,0.5
title          1.0,1.0,0.5,0.5
*unique_count  2.0,2.0,0.0,0.0


In [22]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py| sort -k1,1 | NaiveBayes/train_reducer_smooth.py | NaiveBayes/postprocess_mapper.py | sort -k1,1 -k2,2 | column -t

A  *total_count   3.0,8.0,0.0,0.0
A  *unique_count  6.0,6.0,0.0,0.0
A  ClassPriors    1.0,3.0,0.25,0.75
A  beijing        0.0,1.0,0.0,0.125
A  chinese        1.0,5.0,0.3333333333333333,0.625
A  japan          1.0,0.0,0.3333333333333333,0.0
A  macao          0.0,1.0,0.0,0.125
A  shanghai       0.0,1.0,0.0,0.125
A  tokyo          1.0,0.0,0.3333333333333333,0.0


In [23]:
# part e - write a systems test for your mapper + reducer together here
!cat NaiveBayes/chineseTrain.txt | NaiveBayes/train_mapper.py| sort -k1,1 | NaiveBayes/train_reducer_smooth.py | NaiveBayes/postprocess_mapper.py | sort -k1,1 -k2,2 | NaiveBayes/postprocess_reducer.py | column -t

beijing      0.0,1.0,0.1111111111111111,0.14285714285714285
chinese      1.0,5.0,0.2222222222222222,0.42857142857142855
japan        1.0,0.0,0.2222222222222222,0.07142857142857142
macao        0.0,1.0,0.1111111111111111,0.14285714285714285
shanghai     0.0,1.0,0.1111111111111111,0.14285714285714285
tokyo        1.0,0.0,0.2222222222222222,0.07142857142857142
ClassPriors  1.0,3.0,0.25,0.75


In [24]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-chinese-NB-model_temp

rm: `/user/root/HW2/smoothed-chinese-NB-model_temp': No such file or directory


In [25]:
# part e - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/smoothed-chinese-NB-model_temp \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob4831795807205008295.jar tmpDir=null
19/09/18 21:13:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 21:13:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 21:13:21 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/18 21:13:21 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 21:13:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568841085575_0002
19/09/18 21:13:21 INFO impl.YarnClientImpl: Submitted application application_1568841085575_0002
19/09/18 21:13:21 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568841085575_0002/
19/09/18 21:13:21 INFO mapreduce.Job: Running job: job_1568841085575_0002
19/09/18 21:13:29 INFO mapreduce.Job: Job job_1568841085575_0002 running in uber mode : false
19/09/18 21:13:29 INFO mapreduce.Job:  map 0% reduce 0%
19/09

In [26]:
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-chinese-NB-model
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/postprocess_mapper.py,NaiveBayes/postprocess_reducer.py \
  -mapper postprocess_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer postprocess_reducer.py \
  -input {HDFS_DIR}/smoothed-chinese-NB-model_temp \
  -output {HDFS_DIR}/smoothed-chinese-NB-model \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

rm: `/user/root/HW2/smoothed-chinese-NB-model': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7838169417323661751.jar tmpDir=null
19/09/18 21:13:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 21:13:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/18 21:13:51 INFO mapred.FileInputFormat: Total input paths to process : 2
19/09/18 21:13:51 INFO mapreduce.JobSubmitter: number of splits:2
19/09/18 21:13:51 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568841085575_0003
19/09/18 21:13:51 INFO impl.YarnClientImpl: Submitted application application_1568841085575_0003
19/09/18 21:13:51 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568841085575_0003/
19/09/18 21:13:51 INFO mapreduce.Job: Running job: job_1568841085575_0003
19/09/18 21:13:59 INFO mapreduce.Job: Job job_1568841085575_0003 running in uber 

In [27]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/smoothed-chinese-NB-model_temp/part-000* > NaiveBayes/smoothed-chinese-NB-model_temp.txt

In [28]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/smoothed-chinese-NB-model_temp.txt | column -t

ClassPriors    1.0,3.0,0.25,0.75
*total_count   3.0,8.0,0.0,0.0
beijing        0.0,1.0,0.0,0.125
japan          1.0,0.0,0.3333333333333333,0.0
tokyo          1.0,0.0,0.3333333333333333,0.0
*unique_count  3.0,3.0,0.0,0.0
ClassPriors    1.0,3.0,0.25,0.75
*total_count   3.0,8.0,0.0,0.0
chinese        1.0,5.0,0.3333333333333333,0.625
macao          0.0,1.0,0.0,0.125
shanghai       0.0,1.0,0.0,0.125
*unique_count  3.0,3.0,0.0,0.0


In [29]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/smoothed-chinese-NB-model/part-000* > NaiveBayes/smoothed-chinese-NB-model.txt

In [30]:
!cat NaiveBayes/smoothed-chinese-NB-model.txt | column -t

beijing      0.0,1.0,0.1111111111111111,0.14285714285714285
chinese      1.0,5.0,0.2222222222222222,0.42857142857142855
japan        1.0,0.0,0.2222222222222222,0.07142857142857142
macao        0.0,1.0,0.1111111111111111,0.14285714285714285
shanghai     0.0,1.0,0.1111111111111111,0.14285714285714285
tokyo        1.0,0.0,0.2222222222222222,0.07142857142857142
ClassPriors  1.0,3.0,0.25,0.75


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance, and draw some conclusions.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. 

[`NOTE:` _Make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]


* __e) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __f) short response:__ What do you notice about the 'top words' we printed in `e`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'e'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q9 Student Answers:
> __d)__ The unsmoothed model's predictions favor heavily on ham (negatives). 19 out of the 20 test examples were predicted to be ham (negative). I'm guessing it's probably because the code I have in `classify_mapper.py`. When making predictions I have these lines of code 
```
    if logpSpam > logpHam:
        pred_class = '1'
    else:
        pred_class = '0'
```
> A document is only predicted as spam if the predicted probability of spam is __strictly larger__ than the predicted probability of ham.  
> Since our training sample size is small, there is going to be a lot of words that occur only in one class but not the other, which will cause us to have many estimated conditional probabilities of zero in a unsmoothed model. As we are summing up the log of probabilities, it might often be the case that each class comes across at least one word that has minus infinite as the log of conditional probablity. As a result, it's often the case that for our unsmoothed model, it's very rare that logpSpam is strictly larger than logpHam.

> In terms of which measure is more important. I think the bottom line is to have high accuracy. However, when deciding between precision and recall trade-off, I think precision is more important than recall. Since we can afford to have some spam sneaking into the inbox, but it might cause more problem when we have important messages being sent to spam folder which might affect the timingly actions people need to take.

> __e)__ We can include all fields into the composite key. Then use `maxClass` to partition and `maxClassP` to sort. Then make sure we use two reducers, so the top ten records in each output file would be the results we need.

> __f)__ 
> Those words are usually common words in English that don't contain class-specific information.
> Increasing k will decrease the probabilities for the top words.
> Increasing k will make the probabilities of words that occur much more in one class than another become more even.
> Increasing k makes the model learn less about the local information that the training set carries; therefore, it reduces the variance of our model. In other words, as k increases, it makes the model more biased towards a more simple model where all conditional probabilities are more even and we are more dependent on the priors to make our predictions.  

__Test/Train split__

In [31]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [81]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -mapper train_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

Deleted /user/root/HW2/enron-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob3766837961684397282.jar tmpDir=null
19/09/19 00:25:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:25:09 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:25:10 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/19 00:25:10 INFO mapreduce.JobSubmitter: number of splits:2
19/09/19 00:25:10 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568841085575_0022
19/09/19 00:25:10 INFO impl.YarnClientImpl: Submitted application application_1568841085575_0022
19/09/19 00:25:10 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568841085575_0022/
19/09/19 00:25:10 INFO mapreduce.Job: Running job: job_1568841085575_0022
19/09/19 00:25:16 INFO mapreduce.Job: Job job_1568841085575_0022 running in uber mode : false
19/09/19 00:25:16 INFO map

In [82]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2.0,4.0,0.0001725476662928134,0.00029682398337785694


In [83]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1.0,22.0,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [84]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -mapper train_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer train_reducer_smooth.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smoothed-model-temp \
  -numReduceTasks 4 \
  -cmdenv PATH={PATH}

!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -files NaiveBayes/postprocess_mapper.py,NaiveBayes/postprocess_reducer.py \
  -mapper postprocess_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer postprocess_reducer.py \
  -input {HDFS_DIR}/smoothed-model-temp \
  -output {HDFS_DIR}/smooth-model \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

!hdfs dfs -rm -r {HDFS_DIR}/smoothed-model-temp

# save the model locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

Deleted /user/root/HW2/smooth-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8074166393501718636.jar tmpDir=null
19/09/19 00:25:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:25:44 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:25:44 WARN security.UserGroupInformation: PriviledgedActionException as:root (auth:SIMPLE) cause:org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://quickstart.cloudera:8020/user/root/HW2/smoothed-model-temp already exists
19/09/19 00:25:44 WARN security.UserGroupInformation: PriviledgedActionException as:root (auth:SIMPLE) cause:org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory hdfs://quickstart.cloudera:8020/user/root/HW2/smoothed-model-temp already exists
19/09/19 00:25:44 ERROR streaming.StreamJob: Error Launching job : Output directory hdfs://quickstart.cloudera:8020/user/root/HW2/smoothed-

In [85]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2.0,4.0,0.0001858045336306206,0.00027730020520215184


In [86]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1.0,22.0,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [87]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [88]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	-8.90668134500626	-8.10769031284611	1
d6	1	-5.780743515794329	-4.179502370564408	1
d7	0	-6.591673732011658	-7.511706880737812	0
d8	0	-4.394449154674438	-5.565796731681498	0
d5	1	-8.90668134500626	-8.10769031284611	 True
d6	1	-5.780743515794329	-4.179502370564408	 True
d7	0	-6.591673732011658	-7.511706880737812	 True
d8	0	-4.394449154674438	-5.565796731681498	 True
# Documents:	4.0
True Positives:	2.0
True Negatives:	2.0
False Positives:	0.0
False Negatives:	0.0
Accuracy:	1.0
Precision:	1.0
Recall:	1.0
F-score:	1.0


In [89]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothed-enron_test-output
# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Unsmoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/unsmoothed-enron_test-output \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/unsmoothed-enron_test-output/part-000* > NaiveBayes/Unsmoothed/results.txt

Deleted /user/root/HW2/unsmoothed-enron_test-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8336377568194787283.jar tmpDir=null
19/09/19 00:26:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:26:28 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:26:29 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/19 00:26:29 INFO mapreduce.JobSubmitter: number of splits:2
19/09/19 00:26:29 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568841085575_0024
19/09/19 00:26:29 INFO impl.YarnClientImpl: Submitted application application_1568841085575_0024
19/09/19 00:26:29 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568841085575_0024/
19/09/19 00:26:29 INFO mapreduce.Job: Running job: job_1568841085575_0024
19/09/19 00:26:37 INFO mapreduce.Job: Job job_1568841085575_0024 running in uber mode : false
19/09/19 

In [90]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-enron_test-output
# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Smoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/smoothed-enron_test-output \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/smoothed-enron_test-output/part-000* > NaiveBayes/Smoothed/results.txt

Deleted /user/root/HW2/smoothed-enron_test-output
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob4150838098583291088.jar tmpDir=null
19/09/19 00:26:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:26:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:26:59 INFO mapred.FileInputFormat: Total input paths to process : 1
19/09/19 00:26:59 INFO mapreduce.JobSubmitter: number of splits:2
19/09/19 00:26:59 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568841085575_0025
19/09/19 00:26:59 INFO impl.YarnClientImpl: Submitted application application_1568841085575_0025
19/09/19 00:26:59 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568841085575_0025/
19/09/19 00:26:59 INFO mapreduce.Job: Running job: job_1568841085575_0025
19/09/19 00:27:05 INFO mapreduce.Job: Job job_1568841085575_0025 running in uber mode : false
19/09/19 00

In [91]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt | column -t
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt | column -t

=========== UNSMOOTHED MODEL ============
#           Documents:          20.0
True        Positives:          11.0
True        Negatives:          2.0
False       Positives:          7.0
False       Negatives:          0.0
Accuracy:   0.65
Precision:  0.6111111111111112
Recall:     1.0
F-score:    0.7586206896551725
=========== SMOOTHED MODEL ============
#           Documents:          20.0
True        Positives:          11.0
True        Negatives:          6.0
False       Positives:          3.0
False       Negatives:          0.0
Accuracy:   0.85
Precision:  0.7857142857142857
Recall:     1.0
F-score:    0.88


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

__`part e starts here:`__

In [78]:
# part e - write your Hadoop job here (sort smoothed model on P(word|class))

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/term-importance-rank
# hadoop job
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=4 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keypartitioner.options="-k3,3" \
  -D mapreduce.partition.keycomparator.options="-k4,4nr" \
  -files NaiveBayes/model_sort_mapper.py \
  -mapper model_sort_mapper.py \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -reducer /bin/cat \
  -input {HDFS_DIR}/smooth-model \
  -output {HDFS_DIR}/term-importance-rank \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

Deleted /user/root/HW2/term-importance-rank
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob807870306961255242.jar tmpDir=null
19/09/19 00:01:56 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:01:56 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/09/19 00:01:57 INFO mapred.FileInputFormat: Total input paths to process : 2
19/09/19 00:01:57 INFO mapreduce.JobSubmitter: number of splits:3
19/09/19 00:01:57 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1568841085575_0021
19/09/19 00:01:57 INFO impl.YarnClientImpl: Submitted application application_1568841085575_0021
19/09/19 00:01:57 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1568841085575_0021/
19/09/19 00:01:57 INFO mapreduce.Job: Running job: job_1568841085575_0021
19/09/19 00:02:04 INFO mapreduce.Job: Job job_1568841085575_0021 running in uber mode : false
19/09/19 00:02:04 

In [79]:
# part e - print top words in each class
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/term-importance-rank/part-0000{idx} | head -10 | column -t


===== part-00000=====

ClassPriors  47.0,33.0,0.5875,0.4125                                ham  0.5875
ect          378.0,0.0,0.023473306082001735,5.546004104043037e-05   ham  0.023473306082001735
and          258.0,277.0,0.01604112473677691,0.015417891409239643   ham  0.01604112473677691
hou          203.0,0.0,0.0126347082868822,5.546004104043037e-05     ham  0.0126347082868822
in           160.0,157.0,0.009971509971509971,0.008762686484387999  ham  0.009971509971509971
for          148.0,153.0,0.00922829183698749,0.008540846320226277   ham  0.00922829183698749
on           122.0,95.0,0.007617985878855444,0.005324163939881316   ham  0.007617985878855444
enron        116.0,0.0,0.007246376811594203,5.546004104043037e-05   ham  0.007246376811594203
i            113.0,106.0,0.007060572277963582,0.00593422439132605   ham  0.007060572277963582
will         113.0,69.0,0.007060572277963582,0.003882202872830126   ham  0.007060572277963582
cat: Unable to write to output stream.

===== part-000

### Congratulations, you have completed HW2! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/1S3P9be24hLspAAPuXINUgMae1v-pUZrmDRmTQ2cwPo0/edit